In [2]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# print path to data
print(os.listdir("/data/dssg/occrp/data/processed_clean"))

['shipping-receipts', 'transcripts', 'bank-statements', 'company-registry', 'court-documents', 'gazettes', 'invoices', 'passport-scan', 'receipts', 'contracts', 'middle-page']


In [3]:
# load images

data_generator = ImageDataGenerator(
    horizontal_flip=True,
    width_shift_range=0.4,
    height_shift_range=0.4,
    zoom_range=0.3,
    rotation_range=20,
)

image_size = 227
batch_size = 10
train_generator = data_generator.flow_from_directory(
    "/data/dssg/occrp/data/processed_clean",
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical",
)

num_classes = len(train_generator.class_indices)

Found 13416 images belonging to 11 classes.


In [4]:
im_size = 227

model = keras.models.Sequential()
model.add(
    tf.keras.applications.efficientnet.EfficientNetB7(
        include_top=False, input_shape=(im_size, im_size, 3), pooling="avg", weights=None
    ),
)
model.add(tf.keras.layers.Dense(11, activation="softmax"))

2022-08-09 16:40:07.308070: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-09 16:40:07.310238: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-09 16:40:07.319495: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-08-09 16:40:07.319524: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries

In [5]:
# compile model
model.compile(
    optimizer=tf.optimizers.SGD(learning_rate=0.001, momentum=0.9), loss="categorical_crossentropy", metrics=["accuracy"]
)

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb7 (Functional)  (None, 2560)             64097687  
                                                                 
 dense (Dense)               (None, 11)                28171     
                                                                 
Total params: 64,125,858
Trainable params: 63,815,131
Non-trainable params: 310,727
_________________________________________________________________


In [14]:
# fit model
count = sum([len(files) for r, d, files in os.walk("/data/dssg/occrp/data/processed_clean")])

model.fit_generator(train_generator, steps_per_epoch=int(count / batch_size) + 1, epochs=10)

Epoch 1/10


/tmp/ipykernel_12096/3988897301.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


   5/1342 [..............................] - ETA: 13:38 - loss: 2.3740 - accuracy: 0.1000

KeyboardInterrupt: 

In [13]:
im_size = 227
model = tf.keras.applications.efficientnet.EfficientNetB7(
    include_top=False, input_shape=(im_size, im_size, 3), pooling="avg", weights=None, classes=11
)

In [10]:
model.layers[-10:-1]

In [14]:
model.outputs

[<KerasTensor: shape=(None, 2560) dtype=float32 (created by layer 'avg_pool')>]

In [12]:
model.inputs

[<KerasTensor: shape=(None, 227, 227, 3) dtype=float32 (created by layer 'input_2')>]